# Salvar imagens da ficha de insumo


In [ ]:
# STEP 1
# import libraries
import pandas as pd
import fitz
import io
from PIL import Image
from io import BytesIO
import os
import pandas as pd
  

def salvar_imagens_pdf_sinapi(path):
    path_images = (os.path.join(path, 'Imagens'))
    isExist = os.path.exists(path_images)

    if not isExist:

      # Create a new directory because it does not exist 
      os.makedirs(path_images)

    # file = "Arquivos A a G.pdf"

    df = pd.DataFrame(columns=['nome_imagem', 'pagina', 'arquivo'])#, 'info_image'])

    counter=0
    for file in ["Arquivos A a G.pdf", "Arquivos H a Z.pdf"]:

        path_file = os.path.join(path, file)

        # open the file
        pdf_file = fitz.open(path_file)



        # STEP 3
        # iterate over PDF pages

        counter_image=1
        previous_image=[0]

        for page_index in range(len(pdf_file)):

            # get the page itself
            page = pdf_file[page_index]
            image_list = page.get_images()
        #     print(len(image_list))

    #         # printing number of images found in this page
    #         if image_list:
    #             print(f"[+] Found a total of {len(image_list)} images in page {page_index+1}")
    #         else:
    #             print("[!] No images found on page", page_index+1)


            for image_index, img in enumerate(page.get_images(), start=1):
#                 print('???',img[7])
                #print(img)


                if img[7]!='Image6':
                    # get the XREF of the image
                    xref = img[0]

                    # extract the image bytes
                    base_image = pdf_file.extract_image(xref)
                    image_bytes = base_image["image"]
            #         display(image.show())

                    stream = BytesIO(image_bytes)

                    image = Image.open(stream).convert("RGB")
                    stream.close()
        #             image.show()

                    if image != previous_image:
    # #                     print ("Identical")
    #                 else:
    # #                     print ("Different")
                        counter_image +=1


                    # get the image extension
                    image_ext = base_image["ext"]
        #             print(image_ext)

                    pag_pdf = page_index+1
        #             print("{:04d}".format(pag_pdf), file[:-4])

                    id_file = "_".join(file.split('.')[0].split()[1:])

                    nome_imagem = id_file+'_imagem_'+"{:04d}".format(counter_image)

                    filename = nome_imagem+'.'+image_ext  #"{:04d}".format(pag_pdf)+'_'+ file[:-4]+'.'+image_ext
                    pagina_ = "{:04d}".format(pag_pdf)


                    df.loc[counter, :]= [nome_imagem, pagina_, file[:-4]]#, list(image.getdata())]
                    counter+=1
    #                 print(counter)



                    #image.save(os.path.join(path,'Imagens', filename))
                    display(image)
                    previous_image=image

    df.to_csv(os.path.join(path,'Imagens', 'A_A_df_imagens_referencia.csv'))
    return df

In [ ]:
# STEP 2
# file path you want to extract images from
path = r"C:\Users\ingri\OneDrive\Desktop\contratao\sinapi_dezembro_2021_desonerado"

df = salvar_imagens_pdf_sinapi(path)
display(df)

# Transformar ficha de insumo em csv

In [ ]:
import textract
import os
import PyPDF2
import re
import pandas as pd

# #ficha com marcas
# colunas = ['Código do SINAPI', 'Descrição Básica', 'Unidade de Cálculo', 'Unidade de Comercialização', 
#            'Normas Técnicas', 'Imagem', 'Informações Gerais',  'Referencial / Parâmetro de Pesquisa', 
#            'Atualizado em', 'Obs']



def ficha_insumos_to_csv(path):
    
    data_sinapi = "_".join(path.split('\\')[-1].split('_')[1:3]) # exemplo = 'dezembro_2021'
    
    #fichas mais recentes 10/2021
    colunas = ['Código do SINAPI', 'Descrição Básica', 'Unidade de Cálculo', 
               'Normas Técnicas', 'Imagem', 'Informações Gerais',  
               'Atualizado em', 'Obs']
    #file = 'Arquivos H a Z'
    df = pd.DataFrame(columns = colunas+['pagina'])
    count=0
    for file in ['Arquivos A a G', 'Arquivos H a Z']:
        pdf_file = open(os.path.join(path, file+'.pdf'), 'rb')
        read_pdf = PyPDF2.PdfFileReader(pdf_file)
        number_of_pages = read_pdf.getNumPages()
#         print(number_of_pages)
#         df = pd.DataFrame(columns = colunas+['pagina'])
#         count=0
        for n_page in range(2, number_of_pages):
            #print(n_page)
            page = read_pdf.getPage(n_page)
            page_content = page.extractText()
            texto = (page_content.replace('\n', ' ').replace("  ", ' '))#.encode('utf-8').decode)
            texto


            for i,coluna in zip(range(len(colunas)), colunas):
#                 print(coluna)
        #         print(colunas[i+1])
                if coluna == colunas[-1]:
                    celula = re.search(coluna+'(.*)', texto).group(1)
                else:
                    celula = re.search(coluna+'(.*)'+colunas[i+1], texto).group(1)
            #     print(celula)
                df.loc[count, coluna] = celula.replace(': ','')
                df.loc[count, 'pagina'] = n_page+1
                df.loc[count,'Arquivo'] = data_sinapi+' - FichaInsumo\\'+file+'.pdf'

            count+=1

        df

        
    df.to_csv(os.path.join(path, 'Arquivos A a Z.csv'))
    return df

In [ ]:
path = r'C:\Users\ingri\OneDrive\Desktop\contratao\sinapi_dezembro_2021_desonerado'
df = ficha_insumos_to_csv(path)
df

# Tabela Sinapi Custo de xls a csv

In [17]:
import pandas as pd
import os

def sinapi_custo_xls_to_csv(file):
    end_file = file.split('\\')[-1][:-3]+'csv' #file com extensão csv e não mais xls
    df = pd.read_excel(file, header = 5)
    df.to_csv(os.path.join("\\".join(file.split('\\')[:-1]), end_file))#'SINAPI_Custo_Ref_Composicoes_Analitico_SC_202111_Desonerado.csv'))
    return df

In [18]:
file = r'C:\Users\ingri\OneDrive\Desktop\contratao\sinapi_dezembro_2021_desonerado\SINAPI_Custo_Ref_Composicoes_Analitico_SC_202112_Desonerado.xls'

df = sinapi_custo_xls_to_csv(file)
df

,DESCRICAO DA CLASSE,SIGLA DA CLASSE,DESCRICAO DO TIPO 1,SIGLA DO TIPO 1,CODIGO DO AGRUPADOR,DESCRICAO DO AGRUPADOR,CODIGO DA COMPOSICAO,DESCRICAO DA COMPOSICAO,UNIDADE,ORIGEM DE PREÇO,...,CUSTO MATERIAL,% MATERIAL,CUSTO EQUIPAMENTO,% EQUIPAMENTO,CUSTO SERVICOS TERCEIROS,% SERVICOS TERCEIROS,CUSTO OUTROS,% OUTROS,% ATRIBUÍDO SÃO PAULO,VINCULO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ASSENTAMENTO DE TUBOS E PECAS,ASTU,FORNEC E/OU ASSENT DE TUBO DE FERRO FUNDIDO JUNTA,45.0,NaN,NaN,97141.0,ASSENTAMENTO DE TUBO DE FERRO FUNDIDO PARA RED...,M,COEFICIENTE DE REPRESENTATIVIDADE,...,"1,13","15,6118143","1,85","25,5977496","0,00","0,0000000","0,00","0,0000000",0.0,CAIXA REFERENCIAL
2,ASSENTAMENTO DE TUBOS E PECAS,ASTU,FORNEC E/OU ASSENT DE TUBO DE FERRO FUNDIDO JUNTA,45.0,NaN,NaN,97141.0,ASSENTAMENTO DE TUBO DE FERRO FUNDIDO PARA RED...,M,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIXA REFERENCIAL
3,ASSENTAMENTO DE TUBOS E PECAS,ASTU,FORNEC E/OU ASSENT DE TUBO DE FERRO FUNDIDO JUNTA,45.0,NaN,NaN,97141.0,ASSENTAMENTO DE TUBO DE FERRO FUNDIDO PARA RED...,M,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIXA REFERENCIAL
4,ASSENTAMENTO DE TUBOS E PECAS,ASTU,FORNEC E/OU ASSENT DE TUBO DE FERRO FUNDIDO JUNTA,45.0,NaN,NaN,97141.0,ASSENTAMENTO DE TUBO DE FERRO FUNDIDO PARA RED...,M,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CAIXA REFERENCIAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44017,SERVICOS DIVERSOS,SEDI,OUTROS,318.0,NaN,NaN,101460.0,VIGIA DIURNO COM ENCARGOS COMPLEMENTARES,MES,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENCARGOS COMPLEMENTARES REFERENCIAL
44018,SERVICOS DIVERSOS,SEDI,OUTROS,318.0,NaN,NaN,101460.0,VIGIA DIURNO COM ENCARGOS COMPLEMENTARES,MES,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENCARGOS COMPLEMENTARES REFERENCIAL
44019,SERVICOS DIVERSOS,SEDI,OUTROS,318.0,NaN,NaN,101460.0,VIGIA DIURNO COM ENCARGOS COMPLEMENTARES,MES,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENCARGOS COMPLEMENTARES REFERENCIAL
44020,SERVICOS DIVERSOS,SEDI,OUTROS,318.0,NaN,NaN,101460.0,VIGIA DIURNO COM ENCARGOS COMPLEMENTARES,MES,COEFICIENTE DE REPRESENTATIVIDADE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENCARGOS COMPLEMENTARES REFERENCIAL
